In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import json
import nibabel as nib
import time

import sys
from tqdm.notebook import tqdm

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

out_path='outputs/AbdomenCTCT/'

/share/data_supergrover1/grossbroehmer/miniconda3/envs/nnunet/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
##preload data

data_path='../Learn2Reg_Dataset_release_v1.0/AbdomenCTCT/'
with open(os.path.join(data_path,'AbdomenCTCT_dataset.json')) as f:
    dataset_info=json.load(f)

val_list=sorted(list(set([x['fixed'] for x in dataset_info['registration_val']] 
              + [x['moving'] for x in dataset_info['registration_val']])))
validation_ = dataset_info['registration_val']
training_ = [x for x in dataset_info['training'] if x['image'] not in val_list]
H,W,D = dataset_info['tensorImageShape']['0']
num_val=len(val_list); num_train=len(training_)
print('Training:',len(training_),'; Validation',len(val_list))


##validation
seg_val = torch.zeros(num_val,H,W,D).long().pin_memory()
img_val = torch.zeros(num_val,1,H//2,W//2,D//2).pin_memory()
t0 = time.time()
for ii,i in enumerate(val_list):
    seg_val[ii] = torch.from_numpy(nib.load(os.path.join(data_path,i.replace('image','label'))).get_fdata()).long()
    img_val[ii] = F.avg_pool3d(torch.from_numpy(nib.load(os.path.join(data_path,i)).get_fdata()).float().cuda().unsqueeze(0).unsqueeze(0)/500,2).cpu()
t1 = time.time()
print('validaion data loaded in %.2f s' % (t1-t0))

FileNotFoundError: [Errno 2] No such file or directory: '../Learn2Reg_Dataset_release_v1.0/AbdomenCTCT/AbdomenCTCT_dataset.json'

In [5]:
### functions 
def jacobian_determinant_3d(dense_flow):
    B,_,H,W,D = dense_flow.size()
    
    dense_pix = dense_flow*(torch.Tensor([H-1,W-1,D-1])/2).view(1,3,1,1,1).to(dense_flow.device)
    gradz = nn.Conv3d(3,3,(3,1,1),padding=(1,0,0),bias=False,groups=3)
    gradz.weight.data[:,0,:,0,0] = torch.tensor([-0.5,0,0.5]).view(1,3).repeat(3,1)
    gradz.to(dense_flow.device)
    grady = nn.Conv3d(3,3,(1,3,1),padding=(0,1,0),bias=False,groups=3)
    grady.weight.data[:,0,0,:,0] = torch.tensor([-0.5,0,0.5]).view(1,3).repeat(3,1)
    grady.to(dense_flow.device)
    gradx = nn.Conv3d(3,3,(1,1,3),padding=(0,0,1),bias=False,groups=3)
    gradx.weight.data[:,0,0,0,:] = torch.tensor([-0.5,0,0.5]).view(1,3).repeat(3,1)
    gradx.to(dense_flow.device)
    #with torch.no_grad():
    jacobian = torch.cat((gradz(dense_pix),grady(dense_pix),gradx(dense_pix)),0)+torch.eye(3,3).view(3,3,1,1,1).to(dense_flow.device)
    jacobian = jacobian[:,:,2:-2,2:-2,2:-2]
    jac_det = jacobian[0,0,:,:,:]*(jacobian[1,1,:,:,:]*jacobian[2,2,:,:,:]-jacobian[1,2,:,:,:]*jacobian[2,1,:,:,:])-\
    jacobian[1,0,:,:,:]*(jacobian[0,1,:,:,:]*jacobian[2,2,:,:,:]-jacobian[0,2,:,:,:]*jacobian[2,1,:,:,:])+\
    jacobian[2,0,:,:,:]*(jacobian[0,1,:,:,:]*jacobian[1,2,:,:,:]-jacobian[0,2,:,:,:]*jacobian[1,1,:,:,:])

    return jac_det

def dice_coeff(outputs, labels, max_label):
    dice = torch.FloatTensor(max_label-1).fill_(0)
    for label_num in range(1, max_label):
        iflat = (outputs==label_num).view(-1).float()
        tflat = (labels==label_num).view(-1).float()
        intersection = torch.mean(iflat * tflat)
        dice[label_num-1] = (2. * intersection) / (1e-8 + torch.mean(iflat) + torch.mean(tflat))
    return dice

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv3d(in_channels,out_channels,3,padding=1,bias=False),\
                                   nn.BatchNorm3d(out_channels),nn.PReLU())
        self.conv2 = nn.Sequential(nn.Conv3d(out_channels,out_channels,1,bias=False),\
                                   nn.BatchNorm3d(out_channels),nn.PReLU())

    def forward(self, x):
        x = self.conv1(x)
        return self.conv2(x)
    
base_ch = 16
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.ModuleDict({'enc1':ConvBlock(64,base_ch*2),'enc2':ConvBlock(base_ch*2,base_ch*3),\
                                      'enc3':ConvBlock(base_ch*3,base_ch*3),'enc4':ConvBlock(base_ch*3,base_ch*4)})
        self.decoder = nn.ModuleDict({'dec1':ConvBlock(base_ch*7,base_ch*3),\
                                      'dec2':ConvBlock(base_ch*6,base_ch*3),'dec3':ConvBlock(base_ch*5,base_ch*2)})
        self.conv1 = ConvBlock(base_ch*2,base_ch*4)
        self.conv2 = nn.Sequential(nn.Conv3d(base_ch*4,base_ch*2,1,bias=False),nn.BatchNorm3d(base_ch*2),nn.PReLU(),\
                                 nn.Conv3d(base_ch*2,base_ch*2,1,bias=False),nn.BatchNorm3d(base_ch*2),nn.PReLU(),\
                                 nn.Conv3d(base_ch*2,3,1))
    def forward(self, x):
        y = []
        upsample = nn.Upsample(scale_factor=2,mode='trilinear')
        for i in range(4):
            x = self.encoder['enc'+str(i+1)](x)
            if(i<3):
                y.append(x)
                x = F.max_pool3d(x,2) 
        for i in range(3):
            x = torch.cat((upsample(x),y.pop()),1)
            x = self.decoder['dec'+str(i+1)](x)
        x = self.conv1(x)
        return F.avg_pool3d(F.avg_pool3d(upsample(self.conv2(x)),5,stride=1,padding=2),5,stride=1,padding=2)
    
class UNet2(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.ModuleDict({'enc1':ConvBlock(1,base_ch*2),'enc2':ConvBlock(base_ch*2,base_ch*3),\
                                      'enc3':ConvBlock(base_ch*3,base_ch*3),'enc4':ConvBlock(base_ch*3,base_ch*4),\
                                      'enc5':ConvBlock(base_ch*4,base_ch*4)})
        self.decoder = nn.ModuleDict({'dec1':ConvBlock(base_ch*8,base_ch*3),'dec2':ConvBlock(base_ch*6,base_ch*3),\
                                      'dec3':ConvBlock(base_ch*6,base_ch*3),'dec4':ConvBlock(base_ch*3,base_ch*2)})
        self.conv1 = ConvBlock(base_ch*2,base_ch*4)
        self.conv2 = nn.Sequential(nn.Conv3d(base_ch*4,base_ch*2,1,bias=False),nn.BatchNorm3d(base_ch*2),nn.PReLU(),\
                                 nn.Conv3d(base_ch*2,base_ch*2,1,bias=False),nn.BatchNorm3d(base_ch*2),nn.PReLU(),\
                                 nn.Conv3d(base_ch*2,32,1))
        self.final = nn.Identity()
    def forward(self, x):
        y = []
        upsample = nn.Upsample(scale_factor=2,mode='trilinear')
        for i in range(5):
            x = self.encoder['enc'+str(i+1)](x)
            if(i<4):
                y.append(x)
                x = F.max_pool3d(x,2) 
        for i in range(4):
            if(i<3):
                x = torch.cat((upsample(x),y.pop()),1)
            x = self.decoder['dec'+str(i+1)](x)
        x = self.conv1(x)
        x = self.conv2(x)
        return self.final(x)

resnet = UNet2()

resnet.cuda()
print()

KeyboardInterrupt: 

In [3]:
models = torch.load('AbdomenCTCT_example_complete.pth')
unet=UNet()
unet.load_state_dict(models['unet'])
resnet.load_state_dict(models['resnet'])

NameError: name 'UNet' is not defined

In [4]:
unet.cuda()
resnet.cuda()
unet.eval()
resnet.eval()
dice45 = torch.zeros(4,45)
t_all = 0
count = 0

val_list_nr=[int(x[-16:-12]) for x in val_list]

for i,val1 in enumerate(val_list_nr):
    for j, val2 in enumerate(val_list_nr):
        if(i>=j):
            continue
        
        torch.cuda.synchronize()
        t0 = time.time()

        
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                feat = resnet(torch.cat((img_val[i:i+1].cuda(),img_val[j:j+1].cuda()),0))
                input = torch.cat((feat[:1],feat[1:2]),1).cuda()

                output = F.interpolate(unet(input),scale_factor=2,mode='trilinear')



        torch.cuda.synchronize()
        t1 = time.time()   
        t_all += (t1-t0)
        with torch.no_grad():
            seg_warped = F.grid_sample(seg_val[j:j+1].unsqueeze(1).float().contiguous().cuda(),output.permute(0,2,3,4,1)+F.affine_grid(torch.eye(3,4).unsqueeze(0).cuda(),(1,1,H,W,D)),mode='nearest')
            jac_det = jacobian_determinant_3d(output[:1])
        d0 = dice_coeff(seg_val[i].cuda().contiguous(),seg_val[j].contiguous().cuda().long(),14).cpu()

        nib_disp_field=((output.permute(0,2,3,4,1) /2)*(torch.tensor([D,W,H]).cuda()-1)).flip(-1).float().squeeze().cpu()
        nib.save(nib.Nifti1Image(nib_disp_field.numpy(), np.eye(4)), os.path.join(out_path, f'disp_{str(val1).zfill(4)}_{str(val2).zfill(4)}.nii.gz'))
        print(f'Saved disp_{str(val1).zfill(4)}_{str(val2).zfill(4)}.nii.gz')

        d1 = dice_coeff(seg_val[i].cuda().contiguous(),seg_warped.contiguous().squeeze().long(),14).cpu()
        #print(d1.mean(),d1)
        dice45[0,count] = d0.mean()
        dice45[1,count] = d1.mean()
        dice45[2,count] = jac_det.std()
        dice45[3,count] = (jac_det<0).float().mean()



        count += 1
    #print(d0.mean(),'after',d1.mean(),jac_det.std(),(jac_det<0).float().mean())

print('%0.4f'%((t_all)/45),'sec/im','Dice before (%)','%0.2f'%(dice45.mean(1)[0].item()*100),\
      'Dice after (%)','%0.2f'%(dice45.mean(1)[1].item()*100),'std(Jac)','%0.4f'%(dice45.mean(1)[2].item()),\
     'neg(Jac)','%0.6f'%(dice45.mean(1)[3].item()))

NameError: name 'unet' is not defined

In [63]:

displacement_fields= '/share/data_zoe3/grossbroehmer/Learn2Reg2022/baselines/outputs/AbdomenCTCT/'
data='/share/data_zoe3/grossbroehmer/Learn2Reg2022/Learn2Reg_Dataset_v1/'
config_files_dir='/share/data_zoe3/grossbroehmer/Learn2Reg2022/L2R/evaluation/evaluation_configs'
output_dir=displacement_fields#'/share/data_zoe3/grossbroehmer/Learn2Reg2022/evaluation/results'
output_suffix='_example.json'
for task in ['AbdomenCTCT']:
    print('Staring', task)
    _i=os.path.join(displacement_fields)
    _d=os.path.join(data,task)
    _o=os.path.join(output_dir,task+output_suffix)
    _c=os.path.join(config_files_dir,task+"_VAL_evaluation_config.json")
    !python /share/data_zoe3/grossbroehmer/Learn2Reg2022/L2R/evaluation/evaluation.py -i {_i} -d {_d} -o{_o} -c{_c} -v
    print(2*'\n')

Staring AbdomenCTCT
/share/data_supergrover1/grossbroehmer/miniconda3/envs/nnunet/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Evaluate 45 cases for: ['LogJacDetStd', 'DSC', 'HD95']
case_results [0]: {'LogJacDetStd': 0.20462614505862706, 'DSC': 0.39922989052677466, 'HD95': 15.941665132646175}
case_results [1]: {'LogJacDetStd': 0.5610970450765086, 'DSC': 0.406835919616467, 'HD95': 13.465253039064285}
case_results [2]: {'LogJacDetStd': 0.32042365233439846, 'DSC': 0.48873730132386256, 'HD95': 10.251239470858536}
case_results [3]: {'LogJacDetStd': 0.15681494660454515, 'DSC': 0.4706736001567782, 'HD95': 11.234598568885923}
case_results [4]: {'LogJacDetStd': 0.5534540337250122, 'DSC': 0.5057861434252648, 'HD95': 13.167800513595797}
case_results [5]: {'LogJacDetSt